# Библиотеки, настройки, загрузка и настройка датафрэйма

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
from difflib import SequenceMatcher as sm
import xlsxwriter
#from google_trans_new import google_translator 
#from tqdm import tqdm
import os

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Series
df_ser=pd.read_csv('/kaggle/input/global-education-statistics/EdStatsSeries.csv')
#Attainment series
att_ser,att_iname=zip(*df_ser[df_ser['Topic']=='Attainment'][['Series Code','Indicator Name']].values)
#df_ser

In [ ]:
# Stats df
df_stats=pd.read_csv('/kaggle/input/global-education-statistics/EdStatsData.csv')
df_rus_stats=df_stats[df_stats['Country Name']=='Russian Federation']
df_stats_cols=df_stats.columns
#df_rus_stats.info()

# Test section

## Простой поиск фич

In [ ]:
# df_f=df_ser['Indicator Name'].copy()

In [ ]:
# def getPairs(combined_list):
#     combined_list.sort()
#     results = list()
#     grouped = set()
#     for x in combined_list:
#         result_group = list()
#         if(grouped.__contains__(x)):
#             continue
#         for name in combined_list:
#             if(sm(None, x, name).ratio() >= 0.80):
#                 result_group.append(name)
#                 grouped.add(name);
#             else:
#                 pass;

#         results.append(result_group)
#     return results;

In [ ]:
#gp = getPairs(df_f.tolist())

In [ ]:
#gp

In [ ]:
# workbook = xlsxwriter.Workbook('feauteres_all.xlsx')
# worksheet = workbook.add_worksheet()
# row, col = 0,0
# for x in gp:
#     for y in x:
#         worksheet.write(row, col, x[col])
#         col=col+1
#     row=row+1
#     col=0
# workbook.close()

In [ ]:
#print(gp[0][0])

#translator.translate(gp[0][0],)

In [ ]:
# translator = google_translator()
# workbook = xlsxwriter.Workbook('feauteres_shortlist.xlsx')
# worksheet = workbook.add_worksheet()
# row, col = 0,0
# for x in tqdm(gp):
#     worksheet.write(row, 0, x[0])
#     worksheet.write(row, 1, translator.translate(str(x[0]),lang_tgt='ru'))
#     row=row+1
# workbook.close()

## Settings

In [ ]:
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

In [ ]:
RF = 'Russian Federation'

In [ ]:
LTPY = 'Ниже, чем в прошлом году'
HTPY = 'Выше, чем в прошлом году'
LTRF = 'Ниже чем в РФ'
HTRF = 'Выше чем в РФ'

In [ ]:
year='2012'

# Анализ

## Средняя прододжительность начального образования

In [ ]:
pop_avg_year_prim=[val for ind,val in enumerate(att_ser[120:150]) if ind%2==0]
pop_avg_year_prim_df=df_rus_stats[df_rus_stats['Indicator Code'].apply(lambda val:val in pop_avg_year_prim)][df_stats_cols[4:]].dropna(axis=1).sum(axis=0)
pop_avg_year_prim_df

In [ ]:
fig,ax=plt.subplots(figsize=(17,3),ncols=1,nrows=1)
colors=['#ff474c' if val<0 else '#89fe05' for val in pop_avg_year_prim_df.diff().values]
ax.bar(pop_avg_year_prim_df.index,pop_avg_year_prim_df.values,color=colors)
ax.set_xlabel('Years')
ax.set_ylabel('Average years of\nprimary schooling completed\nby total population',fontsize=13)
ax.set_title('Total population')

red_patch = mpatches.Patch(color='#ff474c', label='Меньше предыдущего года')
green_patch = mpatches.Patch(color='#89fe05', label='Больше предыдущего года')
fig.legend(handles=[red_patch,green_patch]);


## Коммуникации

In [ ]:
infra_ser,infra_iname=zip(*df_ser[df_ser['Topic']=='Infrastructure: Communications'][['Series Code','Indicator Name']].values)
infra_ser, infra_iname

In [ ]:
#tests
indi_name='Personal computers (per 100 people)'
indi_name='Internet users (per 100 people)'
df_stats.loc[df_stats['Indicator Name']==indi_name].dropna(axis='columns',how='all')#.dropna(thresh=5)

In [ ]:
#

In [ ]:
fig,[ax1,ax2]=plt.subplots(figsize=(18,6),nrows=1,ncols=2)

pc_df=df_rus_stats[df_rus_stats['Indicator Code'].apply(lambda val:val == infra_ser[0])][df_stats_cols[4:]].dropna(axis=1)
cmap = plt.get_cmap('GnBu')
colors=[cmap(i) for i in np.linspace(0, 1, len(pc_df.columns))]
pc_df.iloc[0].plot.bar(ax=ax1,color=colors)
ax1.set_ylabel(infra_iname[0],fontsize=14)

inter_df=df_rus_stats[df_rus_stats['Indicator Code'].apply(lambda val:val == infra_ser[1])][df_stats_cols[4:]].dropna(axis=1)
cmap = plt.get_cmap('GnBu')
colors=[cmap(i) for i in np.linspace(0, 1, len(inter_df.columns))]
inter_df.iloc[0].plot.bar(ax=ax2,color=colors)
ax2.set_ylabel(infra_iname[1],fontsize=14);

## Результаты обучения

In [ ]:
lrout_ser,lrout_iname=zip(*df_ser[df_ser['Topic']=='Learning Outcomes'][['Series Code','Indicator Name']].values)

### Средняя успеваемость

#### В математике

In [ ]:
indi_name='PISA: Mean performance on the mathematics scale'
mean_per_math_scale=df_rus_stats[df_rus_stats['Indicator Code']==[val[0] for val in list(zip(lrout_ser,lrout_iname)) if indi_name in val[1]][0]][df_stats_cols[4:]].dropna(axis=1).iloc[0]
mean_per_math_scale

In [ ]:
colors=['#ff474c' if val<0 else '#89fe05' for val in mean_per_math_scale.to_frame().diff().values]

fig,ax=plt.subplots(figsize=(15,3))
ax.bar(mean_per_math_scale.index,mean_per_math_scale.values,color=colors)
ax.set_ylabel('Средние результаты \n в математике',fontsize=13)

red_patch = mpatches.Patch(color='#ff474c', label=LTPY)
green_patch = mpatches.Patch(color='#89fe05', label=HTPY)
fig.legend(handles=[red_patch,green_patch]);

#### В чтении

In [ ]:
indi_name='PISA: Mean performance on the reading scale'
mean_per_read_scale=df_rus_stats[df_rus_stats['Indicator Code']==[val[0] for val in list(zip(lrout_ser,lrout_iname)) if indi_name in val[1]][0]][df_stats_cols[4:]].dropna(axis=1).iloc[0]
mean_per_read_scale

In [ ]:
colors=['#ff474c' if val<0 else '#89fe05' for val in mean_per_read_scale.to_frame().diff().values]

fig,ax=plt.subplots(figsize=(15,3))
ax.bar(mean_per_read_scale.index,mean_per_read_scale.values,color=colors)
ax.set_ylabel('Средние результаты в\n чтении',fontsize=13)

red_patch = mpatches.Patch(color='#ff474c', label='Lower than previous year')
green_patch = mpatches.Patch(color='#89fe05', label='Higher than previous year')
fig.legend(handles=[red_patch,green_patch]);

#### В науке

In [ ]:
indi_name='PISA: Mean performance on the science scale'
mean_per_sci_scale=df_rus_stats[df_rus_stats['Indicator Code']==[val[0] for val in list(zip(lrout_ser,lrout_iname)) if indi_name in val[1]][0]][df_stats_cols[4:]].dropna(axis=1).iloc[0]
mean_per_sci_scale

In [ ]:
colors=['#ff474c' if val<0 else '#89fe05' for val in mean_per_sci_scale.to_frame().diff().values]

fig,ax=plt.subplots(figsize=(15,3))
ax.bar(mean_per_sci_scale.index,mean_per_sci_scale.values,color=colors)
ax.set_ylabel('Средние результаты в\n науке',fontsize=13)

red_patch = mpatches.Patch(color='#ff474c', label=LTPY)
green_patch = mpatches.Patch(color='#89fe05', label=HTPY)
fig.legend(handles=[red_patch,green_patch]);

## Средний уровень грамотности взрослого населения (2012), по сравнению с другими странами

In [ ]:
indi_name='PIAAC: Mean Adult Literacy Proficiency. Total'
df_stats_lit=df_stats.iloc[df_stats[df_stats['Indicator Name']==indi_name]['2012'].dropna(axis=0,how='all').index][['Country Name','2012']]
df_stats_lit

In [ ]:

rus_val=df_stats_lit[df_stats_lit['Country Name']==RF]['2012'].values[0]
df_stats_lit=df_stats_lit[df_stats_lit['Country Name']!=RF]

colors=['#ff474c' if val<rus_val else '#89fe05' for val in df_stats_lit['2012'].values]

fig,ax=plt.subplots(figsize=(17,3))

red_patch = mpatches.Patch(color='#ff474c', label=LTRF)
green_patch = mpatches.Patch(color='#89fe05', label=HTRF)
ax.legend(handles=[red_patch,green_patch])

ax.bar(df_stats_lit['Country Name'],df_stats_lit['2012'],color=colors)
ax.bar(['Россия'],rus_val)
ax.set_xticklabels(df_stats_lit['Country Name'].tolist()+['Россия'],rotation=90)
ax.set_ylabel('Средний уровень грамотности\n взрослого населения, 2012',fontsize=12);

In [ ]:
#tests
#df_stats_lit=df_stats.iloc[df_stats[df_stats['Indicator Name']==indi_name][year].dropna(axis=0,how='all').index][['Country Name',year]]
indi_name='PIAAC: Mean Adult Literacy Proficiency. Total'
df_stats.loc[df_stats['Indicator Name']==indi_name].dropna(axis='columns',how='all')

## Средний уровень владения счётом взрослого населения (2012), по сравнению с другими странами

In [ ]:
df_stats_num=df_stats.iloc[df_stats[df_stats['Indicator Name']=='PIAAC: Mean Adult Numeracy Proficiency. Total']['2012'].dropna(axis=0,how='all').index][['Country Name','2012']]
#df_stats_num

In [ ]:
rus_val=df_stats_num[df_stats_num['Country Name']==RF]['2012'].values[0]
df_stats_num=df_stats_num[df_stats_num['Country Name']!=RF]

colors=['#ff474c' if val<rus_val else '#89fe05' for val in df_stats_num['2012'].values]

fig,ax=plt.subplots(figsize=(17,3))

red_patch = mpatches.Patch(color='#ff474c', label=LTRF)
green_patch = mpatches.Patch(color='#89fe05', label=HTRF)
ax.legend(handles=[red_patch,green_patch])


ax.bar(df_stats_num['Country Name'],df_stats_num['2012'],color=colors)
ax.bar(['Россия'],rus_val)
ax.set_xticklabels(df_stats_num['Country Name'].tolist()+['Россия'],rotation=90)
ax.set_ylabel('Средний уровень владения счётом\n взрослого населения, 2012',fontsize=12);

In [ ]:
#tests
indi_name='PIAAC: Mean Adult Numeracy Proficiency. Total'
df_stats.loc[df_stats['Indicator Name']==indi_name].dropna(axis='columns',how='all')

## Сравнение взрослого населения РФ по уровню мастерства в решении проблем в высокотехнологичной среде (%). Без опыта работы с компьютером, 2012 г., с другими странами

In [ ]:
df_stats_prob_no_cs=df_stats.iloc[df_stats[df_stats['Indicator Name']=='PIAAC: Adults by proficiency level in problem solving in technology-rich environments (%). No computer experience']['2012'].dropna(axis=0,how='all').index][['Country Name','2012']]
df_stats_prob_no_cs

In [ ]:
#tests
#indi_name='PIAAC: Adults by proficiency level in problem solving in technology-rich environments (%). No computer experience'
#df_test=df_stats.iloc[df_stats[df_stats['Indicator Name']=='PIAAC: Adults by proficiency level in problem solving in technology-rich environments (%). No computer experience']['2012'].dropna(axis=0,how='all').index][['Country Name','2012']]
#df_stats.loc[df_stats['Indicator Name']==indi_name].dropna(axis='columns',how='all').dropna(thresh=5)

In [ ]:
rus_val=df_stats_prob_no_cs[df_stats_prob_no_cs['Country Name']==RF]['2012'].values[0]
df_stats_prob_no_cs=df_stats_prob_no_cs[df_stats_prob_no_cs['Country Name']!=RF]

colors=['#ff474c' if val<rus_val else '#89fe05' for val in df_stats_prob_no_cs['2012'].values]

fig,ax=plt.subplots(figsize=(17,3))

red_patch = mpatches.Patch(color='#ff474c', label=LTRF)
green_patch = mpatches.Patch(color='#89fe05', label=HTRF)
ax.legend(handles=[red_patch,green_patch])


ax.bar(df_stats_prob_no_cs['Country Name'],df_stats_prob_no_cs['2012'],color=colors)
ax.bar([RF],rus_val)
ax.set_xticklabels(df_stats_prob_no_cs['Country Name'].tolist()+['Россия'],rotation=90)
ax.set_ylabel('Сравнение взрослого населения РФ\nпо уровню мастерства в решении\n проблем в высокотехнологичной среде (%).\n Без опыта работы с компьютером,\n 2012 г., с другими странами',fontsize=12);

## Сравнение расходов РФ на образование в% от общих государственных расходов (%) с другими странами

In [ ]:
df_gdp_edu_exp=df_stats.iloc[df_stats[df_stats['Indicator Name']=='Expenditure on education as % of total government expenditure (%)']['2012'].dropna(axis=0).index][['Country Name','2012']].sort_values(by='2012')
df_gdp_edu_exp

In [ ]:
rus_val=df_gdp_edu_exp[df_gdp_edu_exp['Country Name']==RF]['2012'].iloc[0]
colors=['#ff474c' if val<rus_val else '#89fe05' for val in df_gdp_edu_exp['2012'].values]
fig,ax=plt.subplots(figsize=(20,6))
ax.bar(df_gdp_edu_exp['Country Name'].values,df_gdp_edu_exp['2012'].values,color=colors)
ax.set_xticklabels(df_gdp_edu_exp['Country Name'].values,rotation=90)
ax.set_ylabel('Расходы на образование в %\nот остальных государственных расходов (%)',fontsize=13)

red_patch = mpatches.Patch(color='#ff474c', label=LTRF)
green_patch = mpatches.Patch(color='#89fe05', label=HTRF)
fig.legend(handles=[red_patch,green_patch]);